In [1]:
import mido
import queue
import time
import threading

In [2]:
# Function to read MIDI file and add messages to a priority queue
def read_midi_to_queue(midi_file_path, midi_queue, stop_event):
    midi_file = mido.MidiFile(midi_file_path)
    absolute_time = 0  # to keep track of the absolute time in ticks

    for msg in [m for m in midi_file if not m.is_meta]:  # no meta messages
        if stop_event.is_set():  # check if we should stop reading
            break
        print(msg.time)
        absolute_time += msg.time
        print(f"adding message to queue: {absolute_time}, {msg}")
        midi_queue.put(
            (absolute_time, msg)
        )  # add message with its absolute time to the queue

    # midi_queue.put(None)  # signal that we are done reading


# Function to send messages to a MIDI port at the correct times
def send_midi_from_queue(midi_queue: queue.PriorityQueue, midi_out_port, stop_event):
    start_time = time.time()

    while not stop_event.is_set():
        item = midi_queue.get()

        if item is None:  # check for the stop signal
            break

        absolute_time, msg = item
        sleep_time = absolute_time - (time.time() - start_time)

        if sleep_time > 0:
            time.sleep(sleep_time)

        print(f"sending: {msg}")
        midi_queue.task_done()

In [3]:
# Initialize a priority queue and threading events
midi_queue = queue.PriorityQueue()
stop_event = threading.Event()

# Replace 'your_midi_file.mid' with the path to your MIDI file
midi_file_path = "../data/datasets/test/play/20231220-080-01_0000-0005.mid"

# Start the reading thread
reader_thread = threading.Thread(
    target=read_midi_to_queue, args=(midi_file_path, midi_queue, stop_event)
)
reader_thread.start()

# Initialize a MIDI output port
# Replace 'Your MIDI Output Port' with the name of your actual MIDI output port
# with 'Your MIDI Output Port' as midi_out_port:
# Start the sending thread
sender_thread = threading.Thread(
    target=send_midi_from_queue, args=(midi_queue, "Your MIDI Output Port", stop_event)
)
sender_thread.start()

# Wait for the reading thread to finish
reader_thread.join()

# Wait for the sending thread to finish
sender_thread.join()

0
adding message to queue: 0, program_change channel=0 program=0 time=0
0.5
adding message to queue: 0.5, note_on channel=0 note=31 velocity=41 time=0.5
0.109375
adding message to queue: 0.609375, note_off channel=0 note=31 velocity=64 time=0.109375
0.296875
adding message to queue: 0.90625, note_on channel=0 note=43 velocity=23 time=0.296875
0.13020833333333331
adding message to queue: 1.0364583333333333, note_off channel=0 note=43 velocity=64 time=0.13020833333333331
0.22916666666666666
adding message to queue: 1.265625, note_on channel=0 note=38 velocity=39 time=0.22916666666666666
0.06770833333333333
adding message to queue: 1.3333333333333333, note_off channel=0 note=38 velocity=64 time=0.06770833333333333
0.18229166666666666
adding message to queue: 1.515625, note_on channel=0 note=31 velocity=35 time=0.18229166666666666
0.140625
adding message to queue: 1.65625, note_off channel=0 note=31 velocity=64 time=0.140625
0.23958333333333331
adding message to queue: 1.8958333333333333, 

KeyboardInterrupt: 

In [4]:
mido.MidiFile(midi_file_path).print_tracks()

=== Track 0
MetaMessage('track_name', name='20231220-080-01_0000-0005', time=0)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=36, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=36, notated_32nd_notes_per_beat=8, time=0)
Message('program_change', channel=0, program=0, time=0)
Message('note_on', channel=0, note=31, velocity=41, time=96)
Message('note_off', channel=0, note=31, velocity=64, time=21)
Message('note_on', channel=0, note=43, velocity=23, time=57)
Message('note_off', channel=0, note=43, velocity=64, time=25)
Message('note_on', channel=0, note=38, velocity=39, time=44)
Message('note_off', channel=0, note=38, velocity=64, time=13)
Message('note_on', channel=0, note=31, velocity=35, time=35)
Message('note_off', channel=0, note=31, velocity=64, time=27)
Message('note_on', channel=0, note=43, velocity=35, time=46)
Message('note_off', channel=0, note=43, velocity=64, time=30)
Message('note